In [1]:
book = 'hind_swaraj'
Parts = 1
dir_path = './%s/part'% book
image_path = "./community_images/%s/part" %book

In [2]:
from networkx.readwrite import json_graph
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from networkx.algorithms.community import greedy_modularity_communities
import itertools
import community
import os, os.path
import fnmatch
import random as rand
import json
import matplotlib.colors as colors
import matplotlib.cm as cmx


In [3]:
def convert_to_json(G, node_vals, node_positions,file_name,size_dict):
    data1 = json_graph.node_link_data(G)
    color_map = plt.get_cmap('jet')
    node_values = [float(x) / (max(node_vals) + 1)  for x in node_vals]
    cNorm  = colors.Normalize(vmin=0, vmax=1)
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=color_map)
    """print('color_map')
    print(type(color_map))
    print color_map
    print('node_values')"""
    #print(type(node_values))
    #print node_values
    """
    print('node positions')
    print(type(node_positions))
    print node_positions"""
    my_data1 = {}
    my_data1["nodes"] = []
    my_data1["edges"] = data1["links"]
    idx = 0
    for node in G.nodes():
        print node
        node_dict = {}
        node_dict["id"] = node
        node_dict["x"] = 900 * node_positions[node][0]
        node_dict["y"] = 900 * node_positions[node][1]
        node_dict["size"] = int(1 + 20*size_dict[node])
        node_dict["label"] = node
        #print("idx:" + str(idx))
        #print("node_value:" + str(node_values[idx]))
        
        colorVal = scalarMap.to_rgba(node_values[idx])
        #print("color valo"+str(colorVal[0]) + " coloVal1:" + str(colorVal[1]) + " colorVal2:" + str(colorVal[2]))
        node_dict["color"] = "rgb(" + str(int(colorVal[0] * 255)) + "," + str(int(colorVal[1] * 255)) + "," + str(int(colorVal[2] * 255)) + ")"
        my_data1["nodes"].append(node_dict)
        idx += 1
    for edge_dict in my_data1["edges"]:
        edge_dict["id"] = edge_dict["source"] + "_" + edge_dict["target"]
    with open("community_images/json_graphs/" + file_name + ".json", "w") as write_file:
        json.dump(my_data1, write_file,indent = 1)

In [4]:
import infomap
import sys
reload(sys)
sys.setdefaultencoding('utf8')

ImportError: No module named infomap

In [ ]:
# pip install infomap

for part in range(1,Parts+1):
    for top_percent in [100]:
        image_part_path = image_path + str(part) + "/"
        x = 1
        no_of_chapters = len(fnmatch.filter(os.listdir(dir_path + str(part) + '/'), '*.txt'))
        print('no.of_chapters:' + str(no_of_chapters))
        print('part:' + str(part))
        
          
        for graph_type in ["place","person"]:
            graph_file_path = 'graphs/graphs_part_wise_tfidf/%s_graph_' % book + graph_type + '_part' + str(part) + 'top_tfidf' + str(top_percent) +'.xml'

            G = nx.read_graphml(graph_file_path)
            # Building Infomap network from a NetworkX graph..."
            d,dinv = {},{}
            i = 0
            for n in G.nodes():
                d[n] = i
                dinv[i]=n
                i+=1
            myInfomap = infomap.Infomap("--two-level")

            for e in G.edges():
                #print e
                myInfomap.addLink(d[e[0]],d[e[1]])

            # Find communities with Infomap...
            myInfomap.run()
            #print myInfomap.getModules()

            communities = {}
            nodes = myInfomap.getModules()
            for node in nodes:
                communities[dinv[node]]=nodes[node]
            nx.set_node_attributes(G, name='community', values=communities)
            values = [communities[n] for n in G.nodes()]
            nx.write_graphml(G, 'graphs/graphs_communities/%s_graph_' % book  + graph_type + '_part' + str(part) + 'top_tfidf' + str(top_percent) +'.xml')

            #drawNetwork(G,image_part_path, graph_type, top_percent, communities)
#             plt.figure(figsize=(30, 30))

#             nx.draw_networkx(G, cmap = plt.get_cmap('Pastel1'), node_color = values, with_labels=True,
#                              pos = nx.spring_layout(G, k=0.3*1/np.sqrt(len(G.nodes())),
#                              iterations=20), font_weight='bold', font_family = 'serif',edge_color = '#acff9d',node_size = 3000,font_size=10)
#             pos = nx.spring_layout(G, k=0.3*1/np.sqrt(len(G.nodes())))

            #plt.savefig('./community_images/%s' % book + graph_type + '_part' + str(part) + '_top_tfidf' + str(top_percent) + "_infomap.png")
            #plt.close()

In [ ]:
# graph drawing: not being used for now.
def drawNetwork(G, image_part_path, graph_type, top_percent,communities):
    plt.figure(figsize=(30, 30))
    # position map
    pos = nx.spring_layout(G)
    # community ids
    c = [v for k,v in nx.get_node_attributes(G, 'community').items()]
    numCommunities = max(c) + 1
    # color map from http://colorbrewer2.org/
    cmapLight = colors.ListedColormap(['#a6cee3', '#b2df8a', '#fb9a99', '#fdbf6f', '#cab2d6'], 'indexed', numCommunities)
    cmapDark = colors.ListedColormap(['#1f78b4', '#33a02c', '#e31a1c', '#ff7f00', '#6a3d9a'], 'indexed', numCommunities)

    # edges
    nx.draw_networkx_edges(G, pos)

    # nodes
    nodeCollection = nx.draw_networkx_nodes(G,
        pos = pos,
        node_color = c,
        cmap = cmapLight
    )
    # set node border color to the darker shade
    darkColors = [cmapDark(v) for v in c]
    nodeCollection.set_edgecolor(darkColors)

    # Print node labels separately instead
    for n in G.nodes:
        plt.annotate(n,
            xy = pos[n],
            textcoords = 'offset points',
            horizontalalignment = 'center',
            verticalalignment = 'center',
            xytext = [0, 2],
            color = cmapDark(communities[n])
        )
    
    plt.axis('off')

    plt.savefig(image_part_path + graph_type + '_top_tfidf' + str(top_percent) + "_infomap.png")
    plt.close()

In [ ]:


for part in range(1,Parts+1):
    for top_percent in [100]:
        image_part_path = image_path + str(part) + "/"
        x = 1
        no_of_chapters = len(fnmatch.filter(os.listdir(dir_path + str(part) + '/'), '*.txt'))
        print('no.of_chapters:' + str(no_of_chapters))
        print('part:' + str(part))
        while x <= no_of_chapters:
            #target_x = min(x+16,no_of_chapters+1)
            target_x = no_of_chapters+1
            for graph_type in ["place","person"]:
                graph_file_path = 'graphs/graphs_part_wise_tfidf/graph_' + graph_type + '_part' + str(part) + 'top_tfidf' + str(top_percent) + 'chapters' + str(x) + '-' + str(target_x) +'.xml'
                G = nx.read_graphml(graph_file_path)
                """
                # girvan newman
                communities_generator = nx.algorithms.community.girvan_newman(G)
                top_level_communities = next(communities_generator)
                next_level_communities = next(communities_generator)

                next_level_communities = sorted(map(sorted, next_level_communities))
                values = []
                for node in G.nodes():
                    for i in range(len(next_level_communities)):
                        if node in next_level_communities[i]:
                            values.append(i)

                plt.figure(figsize=(30, 30))
                nx.draw_networkx(G, cmap = plt.get_cmap('jet'), node_color = values, with_labels=True,pos = nx.spring_layout(G, k=0.3*1/np.sqrt(len(G.nodes())), iterations=20))
                plt.savefig(image_part_path + "_girvan_newman.png")
                plt.close()

                # girvan newman with predefined k
                k = 4
                comp = nx.algorithms.community.girvan_newman(G)
                limited = itertools.takewhile(lambda c: len(c) <= k, comp)

                for communities in limited:
                    communities_list = tuple(sorted(c) for c in communities)

                ## we are taking the kth iteration
                communities_k = sorted(map(sorted, communities_list))
                values = []
                for node in G.nodes():
                    for i in range(len(communities_k)):
                        if node in communities_k[i]:
                            values.append(i)

                plt.figure(figsize=(30, 30))
                nx.draw_networkx(G, cmap = plt.get_cmap('jet'), node_color = values, with_labels=True,pos = nx.spring_layout(G, k=0.3*1/np.sqrt(len(G.nodes())), iterations=20))
                plt.savefig(image_part_path + "_girvan_newman_k.png")
                plt.close()
                """
                # python-louvain

                part_graph = community.best_partition(G)
                values = [part_graph.get(node) for node in G.nodes()]
                plt.figure(figsize=(30, 30))
                nx.draw_networkx(G, cmap = plt.get_cmap('Pastel1'), node_color = values, with_labels=True,
                                 pos = nx.spring_layout(G, k=0.3*1/np.sqrt(len(G.nodes())),
                                 iterations=20), font_weight='bold', font_family = 'serif',edge_color = '#acff9d',node_size = 600)
                plt.savefig(image_part_path + graph_type + '_top_tfidf' + str(top_percent) + "_louvian.png")
                plt.close()
                pos = nx.spring_layout(G, k=0.3*1/np.sqrt(len(G.nodes())))
                if top_percent == 100:
                    degree_centrality_nodes = nx.algorithms.centrality.degree_centrality(G)
                    closeness_centrality_nodes = nx.algorithms.centrality.closeness_centrality(G)
                    convert_to_json(G,values,pos,"part_" + str(part) + "_" + graph_type + "_degree",degree_centrality_nodes)
                    convert_to_json(G,values,pos,"part_" + str(part) + "_" + graph_type + "_closeness",closeness_centrality_nodes)
                """
                # greedy modularity

                c = list(greedy_modularity_communities(G))
                c = sorted(map(sorted, c))
                values = []
                for node in G.nodes():
                    for i in range(len(c)):
                        if node in c[i]:
                            values.append(i)

                plt.figure(figsize=(30, 30))
                nx.draw_networkx(G, cmap = plt.get_cmap('jet'), node_color = values, with_labels=True,pos = nx.spring_layout(G, k=0.3*1/np.sqrt(len(G.nodes())), iterations=20))
                plt.savefig(image_part_path + "_greedy_modularity.png")
                plt.close()
                """
            x = target_x